# MPC Weight Tuning with Bayesian Optimization

This notebook uses Optuna for Bayesian optimization to tune MPC economic weights.

**Goal**: Find MPC weights that maximize revenue (evaluated using GA's economic metric)

**Approach**: Bilevel optimization where:
- Outer loop: Optuna searches over MPC weight parameters
- Inner loop: MPC simulation evaluates each weight configuration

In [1]:
# Import necessary modules
import numpy as np
import os
import pandas as pd
import sys
import time
import warnings

import optuna
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate,
    plot_contour
)

here = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.abspath(os.path.join(here, '../../'))
sys.path.insert(0, project_root)

from core.mpc.mpc import MPC
from core.mpc.mpc_params import MPCParams
from core.mpc.mpc_bounds import ControlInputBounds

from core.model.model_carrying_capacities import ModelCarryingCapacities
from core.model.model_disturbances import ModelDisturbances
from core.model.model_growth_rates import ModelGrowthRates
from core.model.model_helpers import get_sim_inputs_from_hourly
from core.model.model_initial_conditions import ModelInitialConditions
from core.model.model_params import ModelParams
from core.model.model_typical_disturbances import ModelTypicalDisturbances
from core.model.model_sensitivities import ModelSensitivities

from core.ga.ga_params import GeneticAlgorithmParams

# Suppress solver output
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

/Users/carlabecker/.pyenv/versions/3.12.2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Model Parameters

Same parameters as the weather scenario experiments for consistency.

In [2]:
# Model parameters (hourly time stepping for MPC)
model_params = ModelParams(
    dt               = 1.0,  # hours/step
    simulation_hours = 2900, # hours
    closed_form      = False,
    verbose          = False
)

# Carrying capacities (corn defaults)
carrying_capacities = ModelCarryingCapacities(
    kh = 3.0,  # m
    kA = 0.65, # m2
    kN = 20,   # number of leaves
    kc = 1000, # number of spikelets
    kP = 0.25  # kg
)

# Growth rates
growth_rates = ModelGrowthRates(
    ah = 0.01,   # 1/hr
    aA = 0.0105, # 1/hr
    aN = 0.011,  # 1/hr
    ac = 0.01,   # 1/hr
    aP = 0.005   # 1/hr
)

# Sensitivities
sensitivities = ModelSensitivities(
    sigma_W = 30,  # hrs
    sigma_F = 300, # hrs
    sigma_T = 30,  # hrs
    sigma_R = 30   # hrs
)

# Initial conditions
initial_conditions = ModelInitialConditions(
    h0=carrying_capacities.kh/model_params.simulation_hours,
    A0=carrying_capacities.kA/model_params.simulation_hours,
    N0=carrying_capacities.kN/model_params.simulation_hours,
    c0=carrying_capacities.kc/model_params.simulation_hours,
    P0=carrying_capacities.kP/model_params.simulation_hours
)

# Typical disturbances
default_typical_disturbances = ModelTypicalDisturbances()
typical_disturbances = ModelTypicalDisturbances(
    typical_water       = default_typical_disturbances.typical_water * model_params.dt,
    typical_fertilizer  = default_typical_disturbances.typical_fertilizer * model_params.dt,
    typical_temperature = default_typical_disturbances.typical_temperature * model_params.dt,
    typical_radiation   = default_typical_disturbances.typical_radiation * model_params.dt
)

# Control bounds
bounds = ControlInputBounds()

print("Model parameters configured.")

Model parameters configured.


## 2. Load Weather Data

In [3]:
# Load baseline weather data
weather_path = "../../io/inputs/hourly_prcp_rad_temp_iowa.csv"
weather_df = pd.read_csv(weather_path)

# Create disturbances
baseline_disturbances = ModelDisturbances(
    precipitation = weather_df['Hourly Precipitation (in)'].to_numpy(),
    radiation     = weather_df['Hourly Radiation (W/m2)'].to_numpy(),
    temperature   = weather_df['Temperature (C)'].to_numpy()
)

print(f"Loaded weather data: {len(weather_df)} hours")
print(f"  Temperature range: {weather_df['Temperature (C)'].min():.1f} to {weather_df['Temperature (C)'].max():.1f} °C")
print(f"  Total precipitation: {weather_df['Hourly Precipitation (in)'].sum():.1f} inches")

Loaded weather data: 2928 hours
  Temperature range: 8.7 to 35.0 °C
  Total precipitation: 31.5 inches


## 3. GA Economic Weights (for Evaluation)

MPC revenue is evaluated using GA's economic weights for fair comparison.

In [4]:
# Get GA economic weights (used for revenue evaluation, NOT MPC objective)
ga_params = GeneticAlgorithmParams()

# These are fixed - used to evaluate the revenue after MPC runs
EVAL_WEIGHT_FRUIT_BIOMASS = ga_params.weight_fruit_biomass  # 4450 $/kg-plant
EVAL_WEIGHT_HEIGHT = ga_params.weight_height                # 35 $/m-plant
EVAL_WEIGHT_LEAF_AREA = ga_params.weight_leaf_area          # 215 $/m2-plant
EVAL_WEIGHT_IRRIGATION = ga_params.weight_irrigation        # 2.0 $/acre-inch
EVAL_WEIGHT_FERTILIZER = ga_params.weight_fertilizer        # 0.614 $/lb-acre

print("GA Economic Weights (for evaluation):")
print(f"  Fruit biomass: ${EVAL_WEIGHT_FRUIT_BIOMASS}/kg-plant")
print(f"  Height:        ${EVAL_WEIGHT_HEIGHT}/m-plant")
print(f"  Leaf area:     ${EVAL_WEIGHT_LEAF_AREA}/m²-plant")
print(f"  Irrigation:    ${EVAL_WEIGHT_IRRIGATION}/acre-inch")
print(f"  Fertilizer:    ${EVAL_WEIGHT_FERTILIZER}/lb-acre")

GA Economic Weights (for evaluation):
  Fruit biomass: $4450/kg-plant
  Height:        $35.0/m-plant
  Leaf area:     $215.0/m²-plant
  Irrigation:    $2.0/acre-inch
  Fertilizer:    $0.614/lb-acre


In [5]:
def compute_revenue(mpc_result: dict) -> float:
    """
    Compute economic revenue using GA weights (for fair comparison).
    
    Revenue = (biomass value + height value + leaf area value) - (irrigation cost + fertilizer cost)
    """
    P_final = mpc_result["P"][-1]
    h_final = mpc_result["h"][-1]
    A_final = mpc_result["A"][-1]
    total_irrigation = np.sum(mpc_result["irrigation"])
    total_fertilizer = np.sum(mpc_result["fertilizer"])
    
    profit = (
        EVAL_WEIGHT_FRUIT_BIOMASS * P_final +
        EVAL_WEIGHT_HEIGHT * h_final +
        EVAL_WEIGHT_LEAF_AREA * A_final
    )
    expenses = (
        EVAL_WEIGHT_IRRIGATION * total_irrigation +
        EVAL_WEIGHT_FERTILIZER * total_fertilizer
    )
    
    return profit - expenses

## 4. Optuna Objective Function

The objective function:
1. Creates MPCParams with trial-suggested weights
2. Runs MPC simulation
3. Computes revenue using GA's evaluation metric
4. Returns revenue (Optuna maximizes this)

In [6]:
def create_objective(disturbances: ModelDisturbances):
    """
    Factory function to create an Optuna objective with fixed disturbances.
    """
    
    def objective(trial: optuna.Trial) -> float:
        """
        Optuna objective function for MPC weight tuning.
        Returns negative revenue (Optuna minimizes by default, we want to maximize).
        """
        
        # Suggest MPC weights
        # Using log scale for cost weights (they tend to span orders of magnitude)
        weight_irrigation = trial.suggest_float('weight_irrigation', 0.001, 10.0, log=True)
        weight_fertilizer = trial.suggest_float('weight_fertilizer', 0.0001, 1.0, log=True)
        
        # Reward weights (linear scale, higher range)
        weight_height = trial.suggest_float('weight_height', 10.0, 1000.0)
        weight_leaf_area = trial.suggest_float('weight_leaf_area', 10.0, 1000.0)
        weight_fruit_biomass = trial.suggest_float('weight_fruit_biomass', 100.0, 10000.0)
        
        # Anomaly penalties (regularization terms)
        weight_water_anomaly = trial.suggest_float('weight_water_anomaly', 0.001, 10.0, log=True)
        weight_fertilizer_anomaly = trial.suggest_float('weight_fertilizer_anomaly', 0.001, 10.0, log=True)
        
        # MPC horizon (optionally tune this too)
        daily_horizon = trial.suggest_int('daily_horizon', 3, 14)
        
        # Create MPC params with suggested weights
        mpc_params = MPCParams(
            daily_horizon           = daily_horizon,
            weight_irrigation       = weight_irrigation,
            weight_fertilizer       = weight_fertilizer,
            weight_height           = weight_height,
            weight_leaf_area        = weight_leaf_area,
            weight_fruit_biomass    = weight_fruit_biomass,
            weight_water_anomaly    = weight_water_anomaly,
            weight_fertilizer_anomaly = weight_fertilizer_anomaly,
            solver                  = "ipopt",
            solver_options          = {
                "tol": 1e-4,
                "acceptable_iter": 100,
                "max_iter": 500,
                "print_level": 0,  # quiet
                "mu_strategy": "adaptive",
                "linear_solver": "mumps",
            }
        )
        
        # Create MPC controller
        mpc = MPC(
            carrying_capacities  = carrying_capacities,
            disturbances         = disturbances,
            growth_rates         = growth_rates,
            initial_conditions   = initial_conditions,
            model_params         = model_params,
            typical_disturbances = typical_disturbances,
            sensitivities        = sensitivities,
            mpc_params           = mpc_params,
            bounds               = bounds
        )
        
        # Run MPC
        try:
            result = mpc.run()
            revenue = compute_revenue(result)
            
            # Store additional metrics for analysis
            trial.set_user_attr('fruit_biomass', result["P"][-1])
            trial.set_user_attr('height', result["h"][-1])
            trial.set_user_attr('leaf_area', result["A"][-1])
            trial.set_user_attr('total_irrigation', np.sum(result["irrigation"]))
            trial.set_user_attr('total_fertilizer', np.sum(result["fertilizer"]))
            
            return revenue
            
        except Exception as e:
            # Return a very low value if MPC fails
            trial.set_user_attr('error', str(e))
            return -1000.0  # Penalize failed runs
    
    return objective

print("Objective function defined.")

Objective function defined.


## 5. Run Bayesian Optimization

In [7]:
'''
# Configuration
N_TRIALS = 100  # Number of Optuna trials (increase for better convergence)
N_STARTUP_TRIALS = 20  # Random trials before Bayesian sampling kicks in

# Create study (maximize revenue)
sampler = optuna.samplers.TPESampler(
    n_startup_trials=N_STARTUP_TRIALS,
    seed=42
)

study = optuna.create_study(
    direction='maximize',
    sampler=sampler,
    study_name='mpc_weight_tuning'
)

# Create objective with baseline weather
objective = create_objective(baseline_disturbances)

print(f"Starting Bayesian optimization with {N_TRIALS} trials...")
print(f"(Each trial runs a full MPC simulation - this may take a while)\n")

# Run optimization with progress callback
def callback(study, trial):
    if trial.number % 10 == 0:
        print(f"Trial {trial.number}: Revenue=${trial.value:.2f}, Best=${study.best_value:.2f}")

t_start = time.time()
study.optimize(objective, n_trials=N_TRIALS, callbacks=[callback], show_progress_bar=True)
t_elapsed = time.time() - t_start

print(f"\nOptimization complete in {t_elapsed/60:.1f} minutes")
'''

'\n# Configuration\nN_TRIALS = 100  # Number of Optuna trials (increase for better convergence)\nN_STARTUP_TRIALS = 20  # Random trials before Bayesian sampling kicks in\n\n# Create study (maximize revenue)\nsampler = optuna.samplers.TPESampler(\n    n_startup_trials=N_STARTUP_TRIALS,\n    seed=42\n)\n\nstudy = optuna.create_study(\n    direction=\'maximize\',\n    sampler=sampler,\n    study_name=\'mpc_weight_tuning\'\n)\n\n# Create objective with baseline weather\nobjective = create_objective(baseline_disturbances)\n\nprint(f"Starting Bayesian optimization with {N_TRIALS} trials...")\nprint(f"(Each trial runs a full MPC simulation - this may take a while)\n")\n\n# Run optimization with progress callback\ndef callback(study, trial):\n    if trial.number % 10 == 0:\n        print(f"Trial {trial.number}: Revenue=${trial.value:.2f}, Best=${study.best_value:.2f}")\n\nt_start = time.time()\nstudy.optimize(objective, n_trials=N_TRIALS, callbacks=[callback], show_progress_bar=True)\nt_elapse

In [8]:
'''
import pickle
import os

os.makedirs('tuning_results', exist_ok=True)
with open('tuning_results/optuna_study.pkl', 'wb') as f:
    pickle.dump(study, f)
print("Study saved!")
'''

'\nimport pickle\nimport os\n\nos.makedirs(\'tuning_results\', exist_ok=True)\nwith open(\'tuning_results/optuna_study.pkl\', \'wb\') as f:\n    pickle.dump(study, f)\nprint("Study saved!")\n'

In [9]:
import pickle
with open('tuning_results/optuna_study.pkl', 'rb') as f:
    study = pickle.load(f)
print(f"Loaded study with {len(study.trials)} trials, best=${study.best_value:.2f}")

Loaded study with 100 trials, best=$1027.10


## 6. Results Analysis

In [10]:
# Best parameters
print("=" * 60)
print("BEST PARAMETERS FOUND")
print("=" * 60)
print(f"\nBest Revenue: ${study.best_value:.2f}")
print(f"\nOptimal MPC Weights:")
for param, value in study.best_params.items():
    if 'weight' in param:
        print(f"  {param}: {value:.6f}")
    else:
        print(f"  {param}: {value}")

# Best trial additional metrics
best_trial = study.best_trial
print(f"\nBest Trial Outcomes:")
print(f"  Fruit biomass:    {best_trial.user_attrs.get('fruit_biomass', 'N/A'):.4f} kg")
print(f"  Height:           {best_trial.user_attrs.get('height', 'N/A'):.4f} m")
print(f"  Leaf area:        {best_trial.user_attrs.get('leaf_area', 'N/A'):.4f} m²")
print(f"  Total irrigation: {best_trial.user_attrs.get('total_irrigation', 'N/A'):.2f} inches")
print(f"  Total fertilizer: {best_trial.user_attrs.get('total_fertilizer', 'N/A'):.2f} lbs")

BEST PARAMETERS FOUND

Best Revenue: $1027.10

Optimal MPC Weights:
  weight_irrigation: 0.464136
  weight_fertilizer: 0.000967
  weight_height: 761.026243
  weight_leaf_area: 454.645905
  weight_fruit_biomass: 1075.029016
  weight_water_anomaly: 0.006421
  weight_fertilizer_anomaly: 0.007810
  daily_horizon: 3

Best Trial Outcomes:
  Fruit biomass:    0.2269 kg
  Height:           2.7538 m
  Leaf area:        0.5955 m²
  Total irrigation: 0.65 inches
  Total fertilizer: 334.91 lbs


In [11]:
# Convert trials to DataFrame for analysis
trials_df = study.trials_dataframe()
trials_df = trials_df.sort_values('value', ascending=False)

print("Top 10 Trials:")
display_cols = ['number', 'value', 'params_weight_irrigation', 'params_weight_fertilizer', 
                'params_weight_fruit_biomass', 'params_daily_horizon']
available_cols = [c for c in display_cols if c in trials_df.columns]
print(trials_df[available_cols].head(10).to_string())

Top 10 Trials:
    number        value  params_weight_irrigation  params_weight_fertilizer  params_weight_fruit_biomass  params_daily_horizon
91      91  1027.099076                  0.464136                  0.000967                  1075.029016                     3
97      97  1022.505094                  0.216126                  0.000645                   163.891150                     4
78      78  1020.432939                  1.000830                  0.001704                  2991.317168                     5
92      92  1017.833368                  0.487009                  0.000978                   855.828759                     3
96      96  1016.885119                  1.193854                  0.000636                   181.881719                     4
85      85  1015.437578                  0.483693                  0.000952                  1343.658251                     3
84      84  1012.166629                  0.486840                  0.000983                  156

In [12]:
# Optimization history
fig = plot_optimization_history(study)
fig.update_layout(
    title='MPC Weight Tuning: Optimization History',
    xaxis_title='Trial Number',
    yaxis_title='Revenue ($)'
)
fig.show()

In [13]:
!pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [14]:
print(f"Study has {len(study.trials)} trials")
print(f"Best value: ${study.best_value:.2f}")

Study has 100 trials
Best value: $1027.10


In [15]:
# Parameter importances
fig = plot_param_importances(study)
fig.update_layout(title='MPC Weight Tuning: Parameter Importances')
fig.show()

In [16]:
# Parallel coordinate plot (shows relationships between parameters)
fig = plot_parallel_coordinate(study)
fig.update_layout(title='MPC Weight Tuning: Parallel Coordinates')
fig.show()

In [17]:
# Contour plots for key parameter pairs
try:
    fig = plot_contour(study, params=['weight_fruit_biomass', 'weight_irrigation'])
    fig.update_layout(title='Revenue Contour: Fruit Biomass vs Irrigation Weights')
    fig.show()
except:
    print("Not enough trials for contour plot")

## 7. Compare Best MPC with Default and GA

In [18]:
# Run MPC with default weights for comparison
default_mpc_params = MPCParams(
    daily_horizon   = 7,
    solver          = "ipopt",
    solver_options  = {
        "tol": 1e-4,
        "acceptable_iter": 100,
        "max_iter": 500,
        "print_level": 0,
        "mu_strategy": "adaptive",
        "linear_solver": "mumps",
    }
)

print("Running MPC with DEFAULT weights...")
mpc_default = MPC(
    carrying_capacities  = carrying_capacities,
    disturbances         = baseline_disturbances,
    growth_rates         = growth_rates,
    initial_conditions   = initial_conditions,
    model_params         = model_params,
    typical_disturbances = typical_disturbances,
    sensitivities        = sensitivities,
    mpc_params           = default_mpc_params,
    bounds               = bounds
)
result_default = mpc_default.run()
revenue_default = compute_revenue(result_default)
print(f"  Revenue: ${revenue_default:.2f}")

# Run MPC with tuned weights
print("\nRunning MPC with TUNED weights...")
tuned_mpc_params = MPCParams(
    daily_horizon           = study.best_params['daily_horizon'],
    weight_irrigation       = study.best_params['weight_irrigation'],
    weight_fertilizer       = study.best_params['weight_fertilizer'],
    weight_height           = study.best_params['weight_height'],
    weight_leaf_area        = study.best_params['weight_leaf_area'],
    weight_fruit_biomass    = study.best_params['weight_fruit_biomass'],
    weight_water_anomaly    = study.best_params['weight_water_anomaly'],
    weight_fertilizer_anomaly = study.best_params['weight_fertilizer_anomaly'],
    solver                  = "ipopt",
    solver_options          = {
        "tol": 1e-4,
        "acceptable_iter": 100,
        "max_iter": 500,
        "print_level": 0,
        "mu_strategy": "adaptive",
        "linear_solver": "mumps",
    }
)
mpc_tuned = MPC(
    carrying_capacities  = carrying_capacities,
    disturbances         = baseline_disturbances,
    growth_rates         = growth_rates,
    initial_conditions   = initial_conditions,
    model_params         = model_params,
    typical_disturbances = typical_disturbances,
    sensitivities        = sensitivities,
    mpc_params           = tuned_mpc_params,
    bounds               = bounds
)
result_tuned = mpc_tuned.run()
revenue_tuned = compute_revenue(result_tuned)
print(f"  Revenue: ${revenue_tuned:.2f}")

Running MPC with DEFAULT weights...
[Daily MPC] Day 1/120, hour index 0
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 2/120, hour index 24
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 3/120, hour index 48
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 4/120, hour index 72
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 5/120, hour index 96
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 6/120, hour index 120
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 7/120, hour index 144
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 8/120, hour index 168
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 9/120, hour index 192
[CFTOC] Solver status: warning, termination: maxIterations
[CFTOC] WARNING: bad solver status, using fallback control plan.
[Daily MPC] Day 10/120, hour index 216
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 11/120, hour inde

In [19]:
# Summary comparison
print("\n" + "=" * 60)
print("PERFORMANCE COMPARISON")
print("=" * 60)

# GA baseline (from weather scenarios if available)
ga_baseline = 1024.46  # Update this with your GA baseline

print(f"\n{'Method':<25} {'Revenue':>12} {'vs GA':>10}")
print("-" * 50)
print(f"{'GA (fixed strategy)':<25} ${ga_baseline:>10.2f} {'—':>10}")
print(f"{'MPC (default weights)':<25} ${revenue_default:>10.2f} {(revenue_default - ga_baseline)/ga_baseline*100:>9.1f}%")
print(f"{'MPC (tuned weights)':<25} ${revenue_tuned:>10.2f} {(revenue_tuned - ga_baseline)/ga_baseline*100:>9.1f}%")
print("-" * 50)
print(f"{'Improvement from tuning':<25} ${revenue_tuned - revenue_default:>10.2f} {(revenue_tuned - revenue_default)/revenue_default*100:>9.1f}%")


PERFORMANCE COMPARISON

Method                         Revenue      vs GA
--------------------------------------------------
GA (fixed strategy)       $   1024.46          —
MPC (default weights)     $    600.45     -41.4%
MPC (tuned weights)       $   1027.10       0.3%
--------------------------------------------------
Improvement from tuning   $    426.65      71.1%


## 8. Save Results

In [20]:
import json
import pickle

# Create output directory
output_dir = 'tuning_results'
os.makedirs(output_dir, exist_ok=True)

# Save best parameters as JSON
best_params_dict = {
    'best_revenue': study.best_value,
    'best_params': study.best_params,
    'best_trial_attrs': dict(study.best_trial.user_attrs),
    'n_trials': len(study.trials),
    'ga_baseline': ga_baseline,
    'default_revenue': revenue_default,
    'tuned_revenue': revenue_tuned
}

with open(f'{output_dir}/best_params.json', 'w') as f:
    json.dump(best_params_dict, f, indent=2)

# Save full study for later analysis
with open(f'{output_dir}/optuna_study.pkl', 'wb') as f:
    pickle.dump(study, f)

# Save trials DataFrame
trials_df.to_csv(f'{output_dir}/all_trials.csv', index=False)

print(f"Results saved to {output_dir}/")
print(f"  - best_params.json: Best parameters and summary")
print(f"  - optuna_study.pkl: Full Optuna study object")
print(f"  - all_trials.csv:   All trial results")

Results saved to tuning_results/
  - best_params.json: Best parameters and summary
  - optuna_study.pkl: Full Optuna study object
  - all_trials.csv:   All trial results


In [21]:
# Print code snippet for using the tuned weights
print("\n" + "=" * 60)
print("CODE SNIPPET: Use these tuned weights in your MPC")
print("=" * 60)
print(f"""
mpc_params = MPCParams(
    daily_horizon           = {study.best_params['daily_horizon']},
    weight_irrigation       = {study.best_params['weight_irrigation']:.6f},
    weight_fertilizer       = {study.best_params['weight_fertilizer']:.6f},
    weight_height           = {study.best_params['weight_height']:.2f},
    weight_leaf_area        = {study.best_params['weight_leaf_area']:.2f},
    weight_fruit_biomass    = {study.best_params['weight_fruit_biomass']:.2f},
    weight_water_anomaly    = {study.best_params['weight_water_anomaly']:.6f},
    weight_fertilizer_anomaly = {study.best_params['weight_fertilizer_anomaly']:.6f},
    solver                  = "ipopt",
    solver_options          = {{
        "tol": 1e-4,
        "acceptable_iter": 100,
        "max_iter": 500,
        "print_level": 0,
        "mu_strategy": "adaptive",
        "linear_solver": "mumps",
    }}
)
""")


CODE SNIPPET: Use these tuned weights in your MPC

mpc_params = MPCParams(
    daily_horizon           = 3,
    weight_irrigation       = 0.464136,
    weight_fertilizer       = 0.000967,
    weight_height           = 761.03,
    weight_leaf_area        = 454.65,
    weight_fruit_biomass    = 1075.03,
    weight_water_anomaly    = 0.006421,
    weight_fertilizer_anomaly = 0.007810,
    solver                  = "ipopt",
    solver_options          = {
        "tol": 1e-4,
        "acceptable_iter": 100,
        "max_iter": 500,
        "print_level": 0,
        "mu_strategy": "adaptive",
        "linear_solver": "mumps",
    }
)



## 9. (Optional) Multi-Scenario Robust Tuning

For weights that work well across different weather conditions (normal, drought, wet, extreme), optimize over multiple scenarios simultaneously. This finds robust weights at the cost of longer optimization time.

In [22]:
# Generate weather scenarios for robust tuning
from core.weather.stochastic_weather import StochasticWeatherGenerator, get_default_scenarios

# Create generator from baseline weather
generator = StochasticWeatherGenerator(weather_df, seed=42)
all_scenarios = get_default_scenarios()

# Use 5 representative scenarios spanning the range of conditions
selected_names = ['normal_1', 'moderate_dry', 'summer_drought', 'wet_year', 'extreme_drought_heat']
selected_scenarios = [s for s in all_scenarios if s.name in selected_names]

# Generate weather for selected scenarios
scenario_weather = {s.name: generator.generate(s) for s in selected_scenarios}
scenario_disturbances = {}
for name, df in scenario_weather.items():
    scenario_disturbances[name] = ModelDisturbances(
        precipitation = df['Hourly Precipitation (in)'].to_numpy(),
        radiation     = df['Hourly Radiation (W/m2)'].to_numpy(),
        temperature   = df['Temperature (C)'].to_numpy()
    )

print(f"Generated {len(scenario_disturbances)} scenarios for robust tuning:")
for name, scenario in zip(selected_names, selected_scenarios):
    print(f"  {name}: extremity={scenario.extremity_index():.2f}")

Generated 5 scenarios for robust tuning:
  normal_1: extremity=0.00
  moderate_dry: extremity=0.80
  summer_drought: extremity=1.70
  wet_year: extremity=1.10
  extreme_drought_heat: extremity=4.97


In [23]:
def multi_scenario_objective(trial: optuna.Trial) -> float:
    """
    Objective that averages revenue across multiple weather scenarios.
    More robust but slower (runs MPC once per scenario per trial).
    """
    # Suggest MPC weights (same ranges as single-scenario)
    weight_irrigation = trial.suggest_float('weight_irrigation', 0.001, 10.0, log=True)
    weight_fertilizer = trial.suggest_float('weight_fertilizer', 0.0001, 1.0, log=True)
    weight_height = trial.suggest_float('weight_height', 10.0, 1000.0)
    weight_leaf_area = trial.suggest_float('weight_leaf_area', 10.0, 1000.0)
    weight_fruit_biomass = trial.suggest_float('weight_fruit_biomass', 100.0, 10000.0)
    weight_water_anomaly = trial.suggest_float('weight_water_anomaly', 0.001, 10.0, log=True)
    weight_fertilizer_anomaly = trial.suggest_float('weight_fertilizer_anomaly', 0.001, 10.0, log=True)
    daily_horizon = trial.suggest_int('daily_horizon', 3, 14)
    
    mpc_params = MPCParams(
        daily_horizon           = daily_horizon,
        weight_irrigation       = weight_irrigation,
        weight_fertilizer       = weight_fertilizer,
        weight_height           = weight_height,
        weight_leaf_area        = weight_leaf_area,
        weight_fruit_biomass    = weight_fruit_biomass,
        weight_water_anomaly    = weight_water_anomaly,
        weight_fertilizer_anomaly = weight_fertilizer_anomaly,
        solver                  = "ipopt",
        solver_options          = {"tol": 1e-4, "max_iter": 500, "print_level": 0}
    )
    
    # Run MPC on all scenarios and collect revenues
    revenues = []
    for name, disturbances in scenario_disturbances.items():
        try:
            mpc = MPC(
                carrying_capacities  = carrying_capacities,
                disturbances         = disturbances,
                growth_rates         = growth_rates,
                initial_conditions   = initial_conditions,
                model_params         = model_params,
                typical_disturbances = typical_disturbances,
                sensitivities        = sensitivities,
                mpc_params           = mpc_params,
                bounds               = bounds
            )
            result = mpc.run()
            revenues.append(compute_revenue(result))
        except Exception as e:
            revenues.append(-1000.0)  # Penalize failed runs
    
    # Store per-scenario results
    for name, rev in zip(scenario_disturbances.keys(), revenues):
        trial.set_user_attr(f'revenue_{name}', rev)
    
    # Return mean revenue (for average-case optimization)
    # Alternative: return min(revenues) for worst-case optimization
    return np.mean(revenues)

print("Multi-scenario objective function defined.")

Multi-scenario objective function defined.


In [24]:
# Run multi-scenario robust optimization
# Note: This takes ~5x longer than single-scenario (one MPC per scenario per trial)

N_ROBUST_TRIALS = 50  # Fewer trials since each is more expensive

sampler_robust = optuna.samplers.TPESampler(n_startup_trials=15, seed=123)
study_robust = optuna.create_study(
    direction='maximize',
    sampler=sampler_robust,
    study_name='mpc_robust_weight_tuning'
)

print(f"Starting ROBUST optimization with {N_ROBUST_TRIALS} trials...")
print(f"Each trial evaluates {len(scenario_disturbances)} weather scenarios.")
print("(This will take a while...)\n")

def robust_callback(study, trial):
    if trial.number % 5 == 0:
        print(f"Trial {trial.number}: Mean Revenue=${trial.value:.2f}, Best=${study.best_value:.2f}")

t_start = time.time()
study_robust.optimize(multi_scenario_objective, n_trials=N_ROBUST_TRIALS, 
                      callbacks=[robust_callback], show_progress_bar=True)
t_elapsed = time.time() - t_start

print(f"\nRobust optimization complete in {t_elapsed/60:.1f} minutes")

Starting ROBUST optimization with 50 trials...
Each trial evaluates 5 weather scenarios.
(This will take a while...)



  0%|          | 0/50 [00:00<?, ?it/s]

[Daily MPC] Day 1/120, hour index 0
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 2/120, hour index 24
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 3/120, hour index 48
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 4/120, hour index 72
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 5/120, hour index 96
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 6/120, hour index 120
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 7/120, hour index 144
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 8/120, hour index 168
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 9/120, hour index 192
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 10/120, hour index 216
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 11/120, hour index 240
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 12/120, hour index 264
[CFTOC] Solver stat

Best trial: 0. Best value: -187.204:   2%|▏         | 1/50 [02:49<2:18:04, 169.06s/it]

[CFTOC] Solver status: ok, termination: optimal
Trial 0: Mean Revenue=$-187.20, Best=$-187.20
[Daily MPC] Day 1/120, hour index 0
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 2/120, hour index 24
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 3/120, hour index 48
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 4/120, hour index 72
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 5/120, hour index 96
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 6/120, hour index 120
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 7/120, hour index 144
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 8/120, hour index 168
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 9/120, hour index 192
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 10/120, hour index 216
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 11/120, hour index 240
[CFTOC] Solv

...

[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 1/120, hour index 0
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 2/120, hour index 24
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 3/120, hour index 48
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 4/120, hour index 72
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 5/120, hour index 96
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 6/120, hour index 120
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 7/120, hour index 144
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 8/120, hour index 168
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 9/120, hour index 192
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 10/120, hour index 216
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 11/120, hour index 240
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC

Best trial: 36. Best value: 859.897:  98%|█████████▊| 49/50 [1:50:46<02:38, 158.44s/it]

[CFTOC] Solver status: warning, termination: maxIterations
[CFTOC] WARNING: bad solver status, using fallback control plan.
[Daily MPC] Day 1/120, hour index 0
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 2/120, hour index 24
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 3/120, hour index 48
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 4/120, hour index 72
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 5/120, hour index 96
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 6/120, hour index 120
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 7/120, hour index 144
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 8/120, hour index 168
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 9/120, hour index 192
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 10/120, hour index 216
[CFTOC] Solver status: ok, termination: optimal
[Daily MPC] Day 11/12

Best trial: 49. Best value: 886.47: 100%|██████████| 50/50 [1:53:21<00:00, 136.03s/it] 

[CFTOC] Solver status: ok, termination: optimal

Robust optimization complete in 113.4 minutes


In [25]:
# Robust optimization results
print("=" * 60)
print("ROBUST TUNING RESULTS (Multi-Scenario)")
print("=" * 60)
print(f"\nBest Mean Revenue: ${study_robust.best_value:.2f}")
print(f"\nOptimal Robust MPC Weights:")
for param, value in study_robust.best_params.items():
    if 'weight' in param:
        print(f"  {param}: {value:.6f}")
    else:
        print(f"  {param}: {value}")

# Show per-scenario performance for best trial
print(f"\nPer-Scenario Revenue (Best Trial):")
best_robust_trial = study_robust.best_trial
for name in scenario_disturbances.keys():
    rev = best_robust_trial.user_attrs.get(f'revenue_{name}', 'N/A')
    if isinstance(rev, float):
        print(f"  {name}: ${rev:.2f}")
    else:
        print(f"  {name}: {rev}")

ROBUST TUNING RESULTS (Multi-Scenario)

Best Mean Revenue: $886.47

Optimal Robust MPC Weights:
  weight_irrigation: 0.176526
  weight_fertilizer: 0.002123
  weight_height: 590.824615
  weight_leaf_area: 490.983764
  weight_fruit_biomass: 1203.308271
  weight_water_anomaly: 2.938806
  weight_fertilizer_anomaly: 1.207217
  daily_horizon: 9

Per-Scenario Revenue (Best Trial):
  normal_1: $1016.79
  moderate_dry: $1058.90
  summer_drought: $977.14
  wet_year: $736.79
  extreme_drought_heat: $642.73


In [26]:
# Save robust study
with open(f'{output_dir}/optuna_study_robust.pkl', 'wb') as f:
    pickle.dump(study_robust, f)

# Save robust best params
robust_params_dict = {
    'best_mean_revenue': study_robust.best_value,
    'best_params': study_robust.best_params,
    'per_scenario_revenue': {name: best_robust_trial.user_attrs.get(f'revenue_{name}') 
                             for name in scenario_disturbances.keys()},
    'n_trials': len(study_robust.trials),
    'scenarios_used': list(scenario_disturbances.keys())
}
with open(f'{output_dir}/best_params_robust.json', 'w') as f:
    json.dump(robust_params_dict, f, indent=2)

print(f"Robust results saved to {output_dir}/")
print(f"  - optuna_study_robust.pkl: Full robust study object")
print(f"  - best_params_robust.json: Best robust parameters")

# Print code snippet
print("\n" + "=" * 60)
print("CODE SNIPPET: Robust MPC weights (work across weather conditions)")
print("=" * 60)
print(f"""
mpc_params = MPCParams(
    daily_horizon           = {study_robust.best_params['daily_horizon']},
    weight_irrigation       = {study_robust.best_params['weight_irrigation']:.6f},
    weight_fertilizer       = {study_robust.best_params['weight_fertilizer']:.6f},
    weight_height           = {study_robust.best_params['weight_height']:.2f},
    weight_leaf_area        = {study_robust.best_params['weight_leaf_area']:.2f},
    weight_fruit_biomass    = {study_robust.best_params['weight_fruit_biomass']:.2f},
    weight_water_anomaly    = {study_robust.best_params['weight_water_anomaly']:.6f},
    weight_fertilizer_anomaly = {study_robust.best_params['weight_fertilizer_anomaly']:.6f},
    solver                  = "ipopt",
    solver_options          = {{
        "tol": 1e-4,
        "acceptable_iter": 100,
        "max_iter": 500,
        "print_level": 0,
        "mu_strategy": "adaptive",
        "linear_solver": "mumps",
    }}
)
""")

Robust results saved to tuning_results/
  - optuna_study_robust.pkl: Full robust study object
  - best_params_robust.json: Best robust parameters

CODE SNIPPET: Robust MPC weights (work across weather conditions)

mpc_params = MPCParams(
    daily_horizon           = 9,
    weight_irrigation       = 0.176526,
    weight_fertilizer       = 0.002123,
    weight_height           = 590.82,
    weight_leaf_area        = 490.98,
    weight_fruit_biomass    = 1203.31,
    weight_water_anomaly    = 2.938806,
    weight_fertilizer_anomaly = 1.207217,
    solver                  = "ipopt",
    solver_options          = {
        "tol": 1e-4,
        "acceptable_iter": 100,
        "max_iter": 500,
        "print_level": 0,
        "mu_strategy": "adaptive",
        "linear_solver": "mumps",
    }
)

